# FinBERT Profiling: DistillBert

This notebook is intentionally **thin**: it reuses the profiling utilities in `pipelines/finBERT/finbert/` (especially `finbert/finbert_profile.py` and `finbert/profile_utils.py`) instead of copying large code blocks.

The purpose of this notebook is to profile finetuning Distilled Bert on financial sentiment analysis.



In [1]:
from __future__ import annotations

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
from finbert.finbert_profile import *
from finbert.profile_utils import get_model_size_mb, print_device_info, setup_nltk_data, timed_eval
import finbert.utils as tools

from finbert.distillFinBert import DistillFinBert


%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

import wandb

✓ Helper utilities loaded


In [2]:
wandb.init(
    entity="si2449-columbia-university",
    project="finbert-experiments",
    name="finetuning-distilled_bert",
    group="knowledge-distillation",
)

wandb: Currently logged in as: shriya-mahakala (aqlab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
cl_path = project_dir/'models'/'distillbert'
cl_data_path = project_dir/'data'/'sentiment_data'

In [4]:
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

config.profile_train_steps = 20

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
finbert = DistillFinBert(config)
finbert.config.base_model = 'distilbert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [6]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

12/16/2025 18:31:51 - INFO - finbert.distillFinBert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [7]:
train_data = finbert.get_data('train')

In [8]:
model = finbert.create_the_model()

In [9]:
start = time.perf_counter()
trained_model = finbert.train(train_examples = train_data, model = model)
train_wall_s = time.perf_counter() - start

12/16/2025 18:31:54 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:31:54 - INFO - finbert.utils -   guid: train-1
12/16/2025 18:31:54 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/16/2025 18:31:54 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:31:54 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:31:54 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/

12/16/2025 18:31:54 - INFO - finbert.distillFinBert -   ***** Loading data *****
12/16/2025 18:31:54 - INFO - finbert.distillFinBert -     Num examples = 3488
12/16/2025 18:31:54 - INFO - finbert.distillFinBert -     Batch size = 32
12/16/2025 18:31:54 - INFO - finbert.distillFinBert -     Num steps = 48
/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/distillFinBert.py:369: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=self.config.use_amp)
Epoch:   0%|          | 0/4 [00:00<?, ?it/s]/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/distillFinBert.py:426: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config.use_amp):
/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/distillFinBert.py:430: Future

Validation losses: [0.9090126248506399]


Iteration: 100%|██████████| 109/109 [00:10<00:00, 10.18it/s]
12/16/2025 18:32:24 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:32:24 - INFO - finbert.utils -   guid: validation-1
12/16/2025 18:32:24 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/16/2025 18:32:24 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:24 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:24 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:24 - INFO - finbert.utils -   label: neutral (id = 2)
12/16/2025 18:32:24 

Validation losses: [0.9090126248506399, 0.4801500668892494]


Iteration: 100%|██████████| 109/109 [00:13<00:00,  8.27it/s]
12/16/2025 18:32:38 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:32:38 - INFO - finbert.utils -   guid: validation-1
12/16/2025 18:32:38 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/16/2025 18:32:38 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:38 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:38 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:38 - INFO - finbert.utils -   label: neutral (id = 2)
12/16/2025 18:32:39 

Validation losses: [0.9090126248506399, 0.4801500668892494, 0.4207475414642921]


Iteration: 100%|██████████| 109/109 [00:13<00:00,  8.26it/s]
12/16/2025 18:32:53 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:32:53 - INFO - finbert.utils -   guid: validation-1
12/16/2025 18:32:53 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/16/2025 18:32:53 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:53 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:53 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:53 - INFO - finbert.utils -   label: neutral (id = 2)
12/16/2025 18:32:53 

Validation losses: [0.9090126248506399, 0.4801500668892494, 0.4207475414642921, 0.40451431618287015]


Epoch: 100%|██████████| 4/4 [00:53<00:00, 13.26s/it]


In [10]:
test_data = finbert.get_data("test")

results = finbert.evaluate(examples=test_data, model=trained_model)

eval_df, eval_timing = timed_eval(
    finbert=finbert, model=trained_model, examples=test_data, use_amp=False
)

12/16/2025 18:32:55 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:32:55 - INFO - finbert.utils -   guid: test-1
12/16/2025 18:32:55 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/16/2025 18:32:55 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:55 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:55 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:55 - IN

Testing: 100%|██████████| 31/31 [00:01<00:00, 24.54it/s]
12/16/2025 18:32:56 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:32:56 - INFO - finbert.utils -   guid: test-1
12/16/2025 18:32:56 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/16/2025 18:32:56 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:56 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:32:56 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [11]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Evaluation Loss:{0:.2f}".format(loss))
    print("Evaluation Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    print(return_val)
    return_val["Evalaution Loss"] = loss
    return_val["Evaluation Accuracy"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    return return_val

In [12]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [13]:
wandb_report = report(results,cols=['labels','prediction','predictions'])

summary = {
        "device": str(finbert.device),
        "model_dir": str(cl_path),
        "train_wall_s": float(train_wall_s),
        "train_examples": int(len(train_data)),
        "train_examples_per_s": float((len(train_data) * finbert.config.num_train_epochs) / train_wall_s)
        if train_wall_s > 0
        else float("inf"),
        "model_size_mb": float(get_model_size_mb(trained_model)),
        "profile_train_steps": finbert.config.profile_train_steps,
        **(finbert.profile_results.get("training_summary", {}) or {}),
        **eval_timing,
        **wandb_report,
    }

wandb.log(summary)
wandb.finish()

/var/tmp/ipykernel_54702/1446198405.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))


Evaluation Loss:0.41
Evaluation Accuracy:0.81

Classification Report:
{'0': {'precision': 0.6984126984126984, 'recall': 0.8239700374531835, 'f1-score': 0.7560137457044673, 'support': 267.0}, '1': {'precision': 0.7160493827160493, 'recall': 0.90625, 'f1-score': 0.8, 'support': 128.0}, '2': {'precision': 0.9107505070993914, 'recall': 0.7808695652173913, 'f1-score': 0.8408239700374532, 'support': 575.0}, 'accuracy': 0.8092783505154639, 'macro avg': {'precision': 0.7750708627427131, 'recall': 0.8370298675568583, 'f1-score': 0.7989459052473068, 'support': 970.0}, 'weighted avg': {'precision': 0.8266103639649431, 'recall': 0.8092783505154639, 'f1-score': 0.8120922194583798, 'support': 970.0}}


Evalaution Loss,▁
Evaluation Accuracy,▁
accuracy,▁
eval_num_samples,▁
eval_samples_per_s,▁
eval_wall_s,▁
model_size_mb,▁
profile_train_steps,▁
train_backward_pass_ms,▁
train_data_transfer_ms,▁
+6,...
